## Twitter API with Tweepy

In [ ]:
import tweepy

In [ ]:
CONSUMER_API_KEY = 'your consumer api key'
CONSUMER_API_SECRET = 'your consumer api secret'
ACCESS_TOKEN ='your access toke'
ACCESS_TOKEN_SECRET = 'your access toke secret'
auth = tweepy.OAuthHandler(CONSUMER_API_KEY, CONSUMER_API_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

In [ ]:
# Test get tweets by any keyword
keywords = ['#dirumahaja'] # put anythin, it could be hashtag, username, or tweet keyword
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
for tweet in tweepy.Cursor(api.search, q=keywords, lang='id').items(10):
    # print in format "user screen name --> tweet text"
    print(f'{tweet.user.screen_name} --> {tweet.text}') 

In [ ]:
# Methods for get tweet in User Timelines
timeline = api.home_timeline()
for tweet in timeline:
    print(f"{tweet.user.name} said {tweet.text}")

In [ ]:
# Methods for Users
# If you want get user's details
user = api.get_user('tweeter username') # put tweeter username here

print('User details: ')
print(user.name)
print(user.description)
print(user.location)

print('Last 20 followers: ')
for follower in user.followers():
    print(follower.name)

In [ ]:
# print all the attribute in the trends_available method
trends_country = api.trends_available()
for trend in trends_country:
    print(trend)

In [ ]:
# Function to get trending topic by country name
def get_trending_topic_by_country(country):
    trends_country = api.trends_available()
    for trend in trends_country:
        if trend['name'] == country:
            country_info = (trend['name'], trend['woeid'])
            print('Country: ', trend['name'], 'WOEID: ', trend['woeid'])
    
    trending_topics = []
    trends_result = api.trends_place(country_info[1])
    for tren in trends_result[0]['trends']:
        print(tren['name'])
        trending_topics.append(tren['name'])
        
    return trending_topics

In [ ]:
trending_topics = get_trending_topic_by_country('Indonesia') # put any country

In [ ]:
# Store tweet attribut into pandas dataframe
import pandas as pd

num_of_tweets = 100 # number of tweets that we want to fetch
# date_since = '2020-03-01'
keywords = trending_topics[0] # get tweet based on Number 1 trending topic 
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

tweets = tweepy.Cursor(api.search, q=keywords, lang='id').items(num_of_tweets)
# These are all I can get with free Twitter API, you can get more with the premium one
dt_tweets = [[tweet.user.name, tweet.user.screen_name, tweet.user.location, 
              tweet.user.verified, tweet.user.followers_count,
              tweet.user.friends_count, tweet.user.listed_count, 
              tweet.created_at, tweet.text, tweet.coordinates, 
              tweet.place, tweet.favorite_count] for tweet in tweets]

# store the tweet into DataFrame for easy read and analysis
df_tweets = pd.DataFrame(data=dt_tweets)
df_tweets.columns = ['username', 'screen_name', 'user_location', 'user_verified', 
                     'followers_count', 'friends_count', 'listed_count', 'tweet_created_at',
                     'tweet_text', 'tweet_coordinates', 'tweet_place', 'tweet_favorite_count']
                     
df_tweets.head()

In [ ]:
df_tweets.sort_values(by='user_location', ascending=False)

In [ ]:
# Create tweet stream listener
# If you want to get tweet stream
class MyStreamListener(tweepy.StreamListener):
    def on_status(self, status):
        print(status.text)
        
mystreamlistener = MyStreamListener()
mystream = tweepy.Stream(auth = api.auth, listener=mystreamlistener)

In [ ]:
mystream.filter(track=['corona'])